# Batch SQL Silver Customer

filter(**Bronze_table**) + **Quarentine_table** => **TEMP_Silver**

**TEMP_Silver** => **Silver_table** OR **Quarentine_table**


## Customer


In [0]:
USE IDENTIFIER(:catalog ||'.'|| :schema)

### Silver Customers

In [0]:
-- Upsert Silver_Customers:
MERGE INTO silver_customers AS T
USING (
  WITH bronze_plus_quarentine AS (
    SELECT 
      Customer_ID,
      Social_Number,
      Name,
      email,
      Address,
      Registration_TimeStamp,
      Operation,
      Moviment_Date
    FROM bronze_customers
    -- Filter by day of moviment (moviment_date):
    WHERE Moviment_Date =:moviment_date

    UNION ALL
    -- Union with customer in quarentine table (wich is expected to be manual updated):
    SELECT
      Customer_ID,
      Social_Number,
      Name,
      email,
      Address,
      Registration_TimeStamp,
      Operation,
      Moviment_Date
    FROM quarentine_customers
  ),

  bronze_plus_quarentine_with_rules AS (
    SELECT *,row_number() OVER(PARTITION BY Customer_ID, Registration_TimeStamp ORDER BY Moviment_Date ASC) AS rn,

            -------------------------- Apply rules: 
            CASE WHEN email IS NOT NULL
                THEN 'valid'
                ELSE 'invalid' 
            END AS status
            ---------------------------------------
    FROM bronze_plus_quarentine
  ) 
  SELECT
      Customer_ID,
      Social_Number,
      Name,
      email,
      Address,
      Registration_TimeStamp,
      Operation,
      Moviment_Date
  FROM bronze_plus_quarentine_with_rules
  ------ We just want to keep the valid records to silver table.
  WHERE (Status = 'valid' OR Operation = 'DELETE') AND rn = 1
  --------------------------------------------------------------------------------------------------------------------------
) AS S
  ON T.Customer_ID = S.Customer_ID AND T.Registration_TimeStamp = S.Registration_TimeStamp
-- INSERT NEW RECORD:
WHEN NOT MATCHED THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
22,0,0,22


### Quarentine Customers

In [0]:
MERGE INTO quarentine_customers AS T
USING (
  WITH bronze_plus_quarentine AS (
    SELECT
      Customer_ID,
      Social_Number,
      Name,
      email,
      Address,
      Registration_TimeStamp,
      Operation,
      Moviment_Date,
      Source_File
    FROM bronze_customers
    -- Filter by day of moviment (moviment_date) and not customer -2 (artificial customer genareted to deal with empty file, meaning: no change in operational database for that day of moviment):
    WHERE Moviment_Date = :moviment_date AND Customer_ID <> -2 AND Operation <> 'DELETE'

    UNION ALL
    -- Union with customer in quarentine table (wich is expected to be manual updated):
    SELECT
      Customer_ID,
      Social_Number,
      Name,
      email,
      Address,
      Registration_TimeStamp,
      Operation,
      Moviment_Date,
      Source_File
    FROM quarentine_customers
  ),

  bronze_plus_quarentine_with_rules AS (
    SELECT *,

            -------------------------- Apply rules: 
            CASE WHEN email IS NOT NULL
                THEN 'valid'
                ELSE 'invalid' 
            END AS status
            ---------------------------------------
    FROM bronze_plus_quarentine
  ) 
  SELECT *
  FROM bronze_plus_quarentine_with_rules
  ------ We just want to keep the invalid records to quarentine table.
  WHERE Status = 'invalid' 
  --------------------------------------------------------------------------------------------------------------------------
) AS S

  ON T.Customer_ID = S.Customer_ID AND T.Registration_TimeStamp = S.Registration_TimeStamp 
-- UPDATE TO LAST RECORD:
WHEN MATCHED THEN UPDATE SET *
-- INSERT NEW RECORD:
WHEN NOT MATCHED THEN INSERT *
-- Deleted retified registers:
WHEN NOT MATCHED BY SOURCE THEN DELETE

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
0,0,0,0


In [0]:
UPDATE IDENTIFIER(:catalog ||'.control.control_tables_moviments')
SET MOVIMENT_DATE = :moviment_date
WHERE Project_Name = :project
  AND Table_name = 'silver_customers'